In [ ]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys


In [ ]:
path = 'RNU_random.txt'
text = open(path).read().lower()
text = text.strip('\n')
test = text[:20000]
text = text[20000:]
print('length:', len(text))

In [ ]:
def chunkstring(string, length):
    return [string[0+i:length+i] for i in range(0, len(string), length)]

In [ ]:
chunk_len = 2
chunk = chunkstring(text,chunk_len)

In [ ]:
hexs = sorted(list(set(chunk)))
print(hexs)

print('total number of hex:', len(hexs))

hex_indices = dict((c, i) for i, c in enumerate(hexs))
indices_hex = dict((i, c) for i, c in enumerate(hexs))

In [ ]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 501
step = 41
sequences = []
next_hex = []
for i in range(0, len(chunk) - maxlen, step):
    sequences.append(text[i*chunk_len: (i + maxlen)*chunk_len])
    next_hex.append(text[(i + maxlen)*chunk_len:(i + maxlen + 1)*chunk_len])
print('number of sequences:', len(sequences))


print('Vectorization...')
X = np.zeros((len(sequences), maxlen, len(hexs)), dtype=np.bool)
y = np.zeros((len(sequences), len(hexs)), dtype=np.bool)
for i, sequence in enumerate(sequences):
    #print ('sentence', sentence)
    for t, char in enumerate(chunkstring(sequence,chunk_len)):        
        X[i, t, hex_indices[char]] = 1
    y[i, hex_indices[next_hex[i]]] = 1

In [ ]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(Convolution1D(nb_filter=maxlen, filter_length=3, border_mode='valid', activation='relu',input_shape=(maxlen, len(hexs))))
model.add(MaxPooling1D(pool_length=2))
model.add(Convolution1D(nb_filter=maxlen*2, filter_length=3, border_mode='valid', activation='relu'))
model.add(MaxPooling1D(pool_length=2))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(len(hexs)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


def sample(preds, temperature=1.0):    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# train the model
for iteration in range(1, 4):    
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=10,  verbose=1)

In [ ]:
# testing with test dataset
text = test
print('corpus length:', len(text))
chunk = chunkstring(text,2)


In [ ]:

step = 1
sequences = []
next_hex = []
for i in range(0, len(chunk) - maxlen, step):
    sequences.append(text[i*2: (i + maxlen)*2])
    next_hex.append(text[(i + maxlen)*2:(i + maxlen + 1)*2])
print('sequences:', len(sequences))


print('Vectorization...')
X = np.zeros((len(sequences), maxlen, len(hexs)), dtype=np.bool)
y = np.zeros((len(sequences), len(hexs)), dtype=np.bool)
for i, sequence in enumerate(sequences):
    #print ('sequences', sequences)
    for t, char in enumerate(chunkstring(sequence,2)):        
        X[i, t, hex_indices[char]] = 1
    y[i, hex_indices[next_hex[i]]] = 1

In [ ]:
n_true = 0
diversity = 1
for i,x in enumerate(X):
    if i % 500 == 0:
        print ("Processed %d %d" % (i,n_true))
    x = x.reshape(1,maxlen,-1)
    preds = model.predict(x, verbose=0)[0]

    next_index = sample(preds, diversity)
    #print (next_index)
    next_hex = indices_hex[next_index]   
    if next_hex == indices_hex[np.argmax(y[i])]:
        n_true += 1
print ("%d %.3f%%" % (n_true,100.0*n_true/len(sequences)))